In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
credit_card_df = pd.read_csv("creditcard.csv")

In [ ]:
credit_card_df.head()

In [ ]:
credit_card_df.describe()

In [ ]:
credit_card_df.info()

In [111]:
credit_card_corr_pre_df = credit_card_df.drop(columns=["Time", "Amount", "Class"], axis=1)
credit_card_corr_df = credit_card_corr_pre_df.corr("pearson")

In [109]:

plt.figure(figsize=(100,100))
sns.heatmap(credit_card_corr_df, annot=True,  fmt=".3f", linewidths=.5, cmap='coolwarm')
plt.show()

#### No correlation in features

In [ ]:
# # Add time in minutes colum to time series
# credit_card_df["Time In Minutes"] = credit_card_df["Time"] / 60.0
# credit_card_df["Time in Hours"] = credit_card_df["Time In Minutes"] / 60.0
# credit_card_df

In [ ]:
# for column in credit_card_df.columns:
#     # Amount will have its own line plot since it is messing up the scale
#     # Class has been removed too
#     if not column in ["Time", "Time In Minutes", "Time in Hours", "Amount", "Class"]:
#         plt.plot(credit_card_df["Time"], credit_card_df[column])

# # plt.plot(credit_card_df["Time"], credit_card_df["V2"], marker="o")
# plt.show()

In [ ]:
# credit_card_time_in_min_df = credit_card_df.groupby(["Time In Minutes"])
# credit_card_time_in_min_df.max()

# # for column in credit_card_df.columns:
# #     # Amount will have its own line plot since it is messing up the scale
# #     # Class has been removed too
# #     if not column in ["Time", "Time In Minutes", "Time in Hours", "Amount", "Class"]:
# #         plt.plot(credit_card_df["Time"], credit_card_df[column])

# # # plt.plot(credit_card_df["Time"], credit_card_df["V2"], marker="o")
# # plt.show()

In [ ]:
# Know the proportions of the feature class
class_proportions_df = pd.DataFrame({
    "count": credit_card_df["Class"].value_counts(),
    "ratio": credit_card_df["Class"].value_counts() / credit_card_df["Class"].count()
})

class_proportions_df

##### The dataset here is very unbalanced

In [ ]:
true_df = credit_card_df[credit_card_df["Class"] == 0]
fraud_df = credit_card_df[credit_card_df["Class"] == 1]

print(true_df.shape, fraud_df.shape)

In [ ]:
true_df_stats = true_df["Amount"].describe()
fraud_df_stats = fraud_df["Amount"].describe()

In [ ]:
true_and_fraud_df_stats = pd.concat(
    [true_df_stats, fraud_df_stats],
    axis=1,
    keys=["true", "fraud"]
    )

from scipy.stats import trim_mean

true_and_fraud_df_stats.loc["trim mean (5%)"] = [
    trim_mean(true_df["Amount"], 0.05),
    trim_mean(fraud_df["Amount"], 0.05)
]

true_and_fraud_df_stats.loc["trim mean (10%)"] = [
    trim_mean(true_df["Amount"], 0.10),
    trim_mean(fraud_df["Amount"], 0.10)
]

true_and_fraud_df_stats

In [ ]:
credit_card_df.drop(columns=["Time", "Class"]).corr("pearson")

In [ ]:
# credit_card_df.boxplot(column="Amount", by="Class")
# plt.show()
sns.boxplot(x="Class", y="Amount", data=credit_card_df)
plt.show()

In [ ]:
credit_card_df[credit_card_df["Amount"] == 0.0].shape

In [ ]:
credit_card_df[credit_card_df["Amount"] > 100.0].shape

In [ ]:
sns.histplot(data=credit_card_df, x="Amount", bins=30)
plt.show()

In [ ]:
sns.histplot(data=true_df, x="Amount", bins=30)
plt.show()

sns.histplot(data=fraud_df, x="Amount", bins=30)
plt.show()

##### The amounts are really small amounts throughout the data set.

##### Undersampling shall be done.

In [ ]:
true_sample_df = true_df.sample(n=fraud_df.shape[0])
true_sample_df.shape

In [ ]:
sample_credit_card_df = pd.concat([true_sample_df, fraud_df])
sample_credit_card_df.shape

In [ ]:
# Split Data
from sklearn.model_selection import train_test_split
X = sample_credit_card_df.drop(columns=["Time", "Class"], axis=1)
y = sample_credit_card_df["Class"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = rf.predict(X_test)

accuracy = accuracy_score(y_pred, y_test)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred, target_names=[str(x) for x in credit_card_df["Class"].unique()]))

#### The precision, recall and f1-scores are really high which is a good thing